In [1]:
import requests

In [2]:
!pip install yelpapi

In [3]:
!pip install pip-system-certs

In [15]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [16]:
# path to your JSON file containing Yelp API credentials
json_file_path = 'F:\\.secret\\yelp_api.json'  

In [26]:
# Read the credentials from the JSON file
with open(json_file_path) as f:
    login = json.load(f)

In [28]:
# Access the Yelp API key from the loaded JSON
yelp_api = YelpAPI(login['api-key'])

In [29]:
LOCATION = '47 W 13th St, New York, NY 10011, USA'
TERM = 'Pizza'

In [30]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/NY_pizza_progress.json"
JSON_FILE

'Data/NY_pizza_progress.json'

In [31]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/NY_pizza_progress.json already exists.


In [32]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [33]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [34]:
## How many results total?
total_results = results['total']
total_results

3300

In [36]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [37]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

165

In [38]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [39]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/165 [00:00<?, ?it/s]

In [40]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)


  0%|          | 0/165 [00:00<?, ?it/s]

HTTPError: 400 Client Error: Bad Request for url: https://api.yelp.com/v3/businesses/search?location=47+W+13th+St%2C+New+York%2C+NY+10011%2C+USA&term=Pizza&offset=1000

In [41]:
# delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [42]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [43]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/NY_pizza_progress.json not found. Saving empty list to new file.
- 0 previous results found.


165

In [44]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/165 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [45]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/PfI8oV...,False,https://www.yelp.com/biz/prince-street-pizza-n...,5087,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72308755605564, 'longitude': -...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1494.758594
1,uc5qQMzs96rzjK27epDCug,joes-pizza-new-york-4,Joe's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/FKjd9R...,False,https://www.yelp.com/biz/joes-pizza-new-york-4...,3073,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.73060076, 'longitude': -74.002...",[delivery],$,"{'address1': '7 Carmine St', 'address2': '', '...",+12123661182,(212) 366-1182,814.694700
2,0UdC0VYX6aAvLiOcpR4MVQ,joes-pizza-new-york-7,Joe's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/QXu6RG...,False,https://www.yelp.com/biz/joes-pizza-new-york-7...,1076,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.73331, 'longitude': -73.98763}","[pickup, delivery]",$,"{'address1': '150 E 14th St', 'address2': '', ...",+12123889474,(212) 388-9474,815.233200
3,16ZnHpuaaBt92XWeJHCC5A,olio-e-più-new-york-7,Olio e Più,https://s3-media4.fl.yelpcdn.com/bphoto/CUpPgz...,False,https://www.yelp.com/biz/olio-e-pi%C3%B9-new-y...,4897,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.733798036104304, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '3 Greenwich Ave', 'address2': No...",+12122436546,(212) 243-6546,413.067370
4,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media3.fl.yelpcdn.com/bphoto/l0Phrn...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,3194,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}",[pickup],$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1522.632411


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,h6qbszNgXgjj9Lrb62m-9w,the-uncommons-new-york,The Uncommons,https://s3-media3.fl.yelpcdn.com/bphoto/23Mnb_...,False,https://www.yelp.com/biz/the-uncommons-new-yor...,424,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"{'latitude': 40.72933, 'longitude': -73.99865}",[delivery],$,"{'address1': '230 Thompson St', 'address2': No...",+16465439215,(646) 543-9215,816.567779
996,uby3glEc0Brt9LAoCI0WTg,l-express-new-york-44,L’Express,https://s3-media3.fl.yelpcdn.com/bphoto/fz2mEn...,False,https://www.yelp.com/biz/l-express-new-york-44...,921,"[{'alias': 'french', 'title': 'French'}, {'ali...",3.5,"{'latitude': 40.7381523, 'longitude': -73.9875...","[delivery, pickup]",$$,"{'address1': '249 Park Ave S', 'address2': '',...",+12122545858,(212) 254-5858,762.369225
997,dhBxIymE-SvvUjstWGyIcw,hasalon-new-york,HaSalon,https://s3-media1.fl.yelpcdn.com/bphoto/nx6pUb...,False,https://www.yelp.com/biz/hasalon-new-york?adju...,141,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",3.0,"{'latitude': 40.764493, 'longitude': -73.992189}",[],NaN,"{'address1': '735 10th Ave', 'address2': '', '...",+12124959024,(212) 495-9024,3144.332996
998,2lbHlWGnpb6AHVfuyZnaPQ,hudson-eats-new-york,Hudson Eats,https://s3-media2.fl.yelpcdn.com/bphoto/ez5FE4...,False,https://www.yelp.com/biz/hudson-eats-new-york?...,15,"[{'alias': 'food_court', 'title': 'Food Court'}]",4.0,"{'latitude': 40.7128272044248, 'longitude': -7...",[],NaN,"{'address1': '225 Liberty St', 'address2': '',...",,,3091.251403
999,Nbam9IMWMgSiQWL8dHql4g,nick-sons-bakery-brooklyn,Nick + Sons Bakery,https://s3-media1.fl.yelpcdn.com/bphoto/1vGcnt...,False,https://www.yelp.com/biz/nick-sons-bakery-broo...,134,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",5.0,"{'latitude': 40.710838, 'longitude': -73.947412}",[delivery],NaN,"{'address1': '205 Leonard St', 'address2': '',...",,,5014.945388


In [47]:
# check for duplicate results
final_df.duplicated().sum()

TypeError: unhashable type: 'list'

In [48]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [49]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [50]:
# save the final results to a compressed csv
final_df.to_csv('Data/NY_pizza_progress.csv.gz', compression='gzip',index=False)
